In [4]:
import re
def occurs_check(var, expr):
    if var == expr:
        return True
    if isinstance(expr, tuple):
        return any(occurs_check(var, sub) for sub in expr)
    return False
def substitute(expr, subst):
    if isinstance(expr, str):
        return subst.get(expr, expr)
    return tuple(substitute(sub, subst) for sub in expr)
def unify(Y1, Y2, subst=None):
    if subst is None:
        subst = {}
    Y1 = substitute(Y1, subst)
    Y2 = substitute(Y2, subst)
    if Y1 == Y2:
        return subst
    if isinstance(Y1, str):
        if occurs_check(Y1, Y2):
            return "FAILURE"
        subst[Y1] = Y2
        return subst
    if isinstance(Y2, str):
        if occurs_check(Y2, Y1):
            return "FAILURE"
        subst[Y2] = Y1
        return subst
    if Y1[0] != Y2[0] or len(Y1) != len(Y2):
        return "FAILURE"
    for a, b in zip(Y1[1:], Y2[1:]):
        subst = unify(a, b, subst)
        if subst == "FAILURE":
            return "FAILURE"
    return subst
def parse_expression(expr_str):
    expr_str = expr_str.strip()
    if '(' not in expr_str:
        return expr_str
    match = re.match(r'([a-zA-Z_]\w*)\((.*)\)', expr_str)
    if not match:
        raise ValueError("Invalid expression format")
    func_name = match.group(1)
    args_str = match.group(2)
    args = []
    depth = 0
    current = ''
    for ch in args_str:
        if ch == ',' and depth == 0:
            args.append(current.strip())
            current = ''
        else:
            if ch == '(':
                depth += 1
            elif ch == ')':
                depth -= 1
            current += ch
    if current:
        args.append(current.strip())
    return (func_name, *[parse_expression(arg) for arg in args])
def expr_to_str(expr):
    if isinstance(expr, str):
        return expr
    func = expr[0]
    args = ', '.join(expr_to_str(a) for a in expr[1:])
    return f"{func}({args})"
def format_mgu(subst):
    if subst == "FAILURE":
        return "❌ Unification Failed"
    if not subst:
        return "✅ Already identical (No substitution needed)"
    return "MGU = { " + ', '.join(f"{var} → {expr_to_str(val)}" for var, val in subst.items()) + " }"
print("Enter first expression (e.g., p(b, X, f(g(Z)))):")
expr1_str = input("expr1 = ")
print("Enter second expression (e.g., p(z, f(Y), f(Y))):")
expr2_str = input("expr2 = ")
expr1 = parse_expression(expr1_str)
expr2 = parse_expression(expr2_str)
result = unify(expr1, expr2)
print("\n--------------------------------------------")
print("Expression 1:", expr_to_str(expr1))
print("Expression 2:", expr_to_str(expr2))
print(format_mgu(result))
print("--------------------------------------------")
print("1BM23CS334")

Enter first expression (e.g., p(b, X, f(g(Z)))):
expr1 = p(b, X, f(g(Z)))
Enter second expression (e.g., p(z, f(Y), f(Y))):
expr2 = p(z, f(Y), f(Y))

--------------------------------------------
Expression 1: p(b, X, f(g(Z)))
Expression 2: p(z, f(Y), f(Y))
MGU = { b → z, X → f(Y), Y → g(Z) }
--------------------------------------------
1BM23CS334
